<a href="https://colab.research.google.com/github/Andbwol/Lenta.Ru-News-Dataset/blob/master/ML_homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import pymorphy2
import joblib as pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [ ]:
df = pd.read_csv('lenta-ru-news.csv')

/Users/18827990/DA_4/lec_venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.shape

(800975, 6)

In [ ]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Нацпроекты             6
Мировой опыт           6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [ ]:
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [ ]:
df.shape

(494804, 6)

In [ ]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [ ]:
df[df.text.isna()]['title'].values[0]

'Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента'

In [ ]:
df.dropna(subset=['text'],inplace = True)

In [ ]:
df.topic.value_counts()

Россия       160442
Мир          136620
Экономика     79528
Спорт         64413
Культура      53796
Name: topic, dtype: int64

In [ ]:
df.topic.value_counts(normalize=True)

Россия       0.324257
Мир          0.276112
Экономика    0.160728
Спорт        0.130180
Культура     0.108723
Name: topic, dtype: float64

In [ ]:
df.isna().sum()

In [ ]:
df[df.text.isna()]['title'].values

In [ ]:
# df = df[~df.text.isna()]

In [ ]:
df.shape

In [ ]:
RANDOM_STATE = 42

### Базовая обработка

In [ ]:
text_transformer = TfidfVectorizer()

In [ ]:
%%time
text = text_transformer.fit_transform(df['text'])

In [ ]:
df['text']

In [ ]:
text.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
%%time
clf = tree.DecisionTreeClassifier()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

### Изменим токенизатор

In [ ]:
stop_russian = stopwords.words('russian')

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
X_train.shape

(395839, 10000)

In [ ]:
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

In [ ]:
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

In [ ]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

In [ ]:
# with open ('transformer.pkl', 'wb') as f:
#     pickle.dump(text_transformer, f)

In [ ]:
# with open ('clf.pkl', 'wb') as f:
#     pickle.dump(clf2, f)

### приведем слова к первой форме

In [ ]:
pattern = r'[^А-Яа-я]+'

In [ ]:
list = [re.sub(pattern, ' ', i) for i in ['проверкаyyy weyweywt iweiwue ><<++ 238238 ? 123 тест']]

In [ ]:
list

['проверка тест']

In [ ]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [ ]:
df['text_clean'] = remove_trash(df['text'])

In [ ]:
df[['text','text_clean']].head()

,text,text_clean
0,"Глава Росстата Владимир Соколин заявил, что в ...",Глава Росстата Владимир Соколин заявил что в я...
1,У российских кинематографистов появится новый...,У российских кинематографистов появится новый ...
2,"Медицинский штаб миланского ""Интера"" обнародов...",Медицинский штаб миланского Интера обнародовал...
3,"Премии ""Книга года"" в 13 номинациях вручены на...",Премии Книга года в номинациях вручены на Моск...
4,Сборная Португалии сыграла вничью с командой И...,Сборная Португалии сыграла вничью с командой И...


In [ ]:
df.isna().sum()

title          0
text           0
topic          0
topic_label    0
text_clean     0
dtype: int64

In [ ]:
# df = df[~df.text_clean.isna()].copy()

In [ ]:
#токенизатор удалил мусор

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic_label'], test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.4297091744411124


### попробуем лемматизацию

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [ ]:
df.text

0         Глава Росстата Владимир Соколин заявил, что в ...
1         У российских кинематографистов  появится новый...
2         Медицинский штаб миланского "Интера" обнародов...
3         Премии "Книга года" в 13 номинациях вручены на...
4         Сборная Португалии сыграла вничью с командой И...
                                ...                        
206632    Бывшей вице-президенту ювелирной компании Tiff...
206633    Минфин предлагает увеличить размер штрафа для ...
206634    Индекс потребительской уверенности россиян в п...
206635    Капитан женской сборной России по теннису Шами...
206636    Союз европейских футбольных ассоциаций (УЕФА) ...
Name: text, Length: 206636, dtype: object

In [ ]:
t = []
text = 'Минфин предлагает увеличить размер штрафа'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [ ]:
" ".join(t)

'минфин предлагать увеличить размер штраф'

In [ ]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

In [ ]:
# пример чтения объекта из файла в оперативную память
with open('df_norma.pkl','rb') as f:
    df = pickle.load(f)

In [ ]:
df.head()

In [ ]:
df[['text','text_clean_normal']].tail(10)

### качество модели после лемматизации

In [ ]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic_label'], test_size=0.20, random_state=RANDOM_STATE)

In [ ]:
# text_transformer.transform(['Минфин предлагает увеличить размер'])

In [ ]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [ ]:
# clf_norm.predict(text_transformer.transform(['Минфин предлагает увеличить размер']))

In [ ]:

print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

In [ ]:


with open('data/tfidf.pkl', 'wb') as f:
    pickle.dump(clf_norm, f)

with open('data/transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

## Сравниваем вектора текстов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


textA = text_transformer.transform(['бывший тренер сборная россия владимир плющев']   )                              

test = cosine_similarity(text_norm, textA)

In [ ]:
testdf = pd.DataFrame(test)
testdf.columns=['vector']

In [ ]:
testdf.sort_values(by='vector',ascending=False).head(5)

In [ ]:
df[df.index==103437]['text_clean_normal'].values